In [17]:
import os
import numpy as np
import pandas as pd
import pandas_ta as ta
from pathlib import Path
from datetime import datetime


downloads_path = str(Path.home() / "Downloads")
tv_exp_fn = f'{downloads_path}\\BINANCE_BTCUSDT.P, 15.csv'
tst_fn = f'{downloads_path}\\tst.csv'

N = 4
per = 144
mult = 1.414
modeLag = False
modeFast = False
beta = 0.00503274062721631
alpha = 0.0954202818890498
lag = 17.875


tv_df = pd.read_csv(tv_exp_fn)
os.remove(tv_exp_fn)
open_price = list(tv_df['open'])
high_price = list(tv_df['high'])
low_price = list(tv_df['low'])
close_price = list(tv_df['close'])

m_src = list(ta.hlc3(tv_df['high'], tv_df['low'], tv_df['close']))
tv_trdata = list(tv_df['tv_trdata'])
m_srcdata = m_src
m_trdata = [idx for idx in range(len(m_src))]

exp_df = pd.DataFrame()
exp_df['tv_trdata'] = tv_trdata
exp_df['m_trdata'] = m_trdata
exp_df['chk'] = [f"=ROUND(A{idx+2},1)=ROUND(B{idx+2},1)" for idx in range(len(open_price))]
exp_df['false_count'] = [f"=COUNTIF(C2:C{len(open_price)}, FALSE)"]+['']*(len(open_price)-1)
exp_df.to_csv(tst_fn, index=False)
os.system(f"start EXCEL.EXE {tst_fn}")


0